In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.cross_validation import train_test_split 
from sklearn.cross_validation import cross_val_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn import preprocessing
from sklearn import metrics 
accuracy = metrics.accuracy_score

/Users/kerrylam/anaconda/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
data = pd.read_csv("http://archive.ics.uci.edu/ml/machine-learning-databases/iris/iris.data", header = None) 
data.columns = ["Sepal Length", "Sepal Width","Petal Length","Petal Width","Class"]
data.head()

,Sepal Length,Sepal Width,Petal Length,Petal Width,Class
0,5.1,3.5,1.4,0.2,Iris-setosa
1,4.9,3.0,1.4,0.2,Iris-setosa
2,4.7,3.2,1.3,0.2,Iris-setosa
3,4.6,3.1,1.5,0.2,Iris-setosa
4,5.0,3.6,1.4,0.2,Iris-setosa


In [3]:
le = preprocessing.LabelEncoder()
le.fit(np.array(data.Class))
data["Class"] = le.transform(data.Class)
data.head()

,Sepal Length,Sepal Width,Petal Length,Petal Width,Class
0,5.1,3.5,1.4,0.2,0
1,4.9,3.0,1.4,0.2,0
2,4.7,3.2,1.3,0.2,0
3,4.6,3.1,1.5,0.2,0
4,5.0,3.6,1.4,0.2,0


In [4]:
X = data.ix[:,0:4]
target = data["Class"]

In [5]:
round(1-(100/len(data)), 4)

0.3333

In [6]:
train, test, targ_train, targ_test = train_test_split(X, target, test_size = round(1-(100/len(data)), 4), random_state = 0)

In [7]:
"""Note: We Will Focus Only On 100 Training Samples For KFolds"""
print ("Train:       ", train.shape)
print ("Train Class: ", targ_train.shape)
print ("Test:        ", test.shape)
print ("Test Class:  ", targ_test.shape)

Train:        (100, 4)
Train Class:  (100,)
Test:         (50, 4)
Test Class:   (50,)


In [8]:
"""Recombine Training Set With Class"""
train = pd.DataFrame.join(train, targ_train)

### Part 1: 

Partition Training Data into 5 Folds:

- Create train_meta with same rows and IDs as training data
- Create Empty Columns: M1 and M2 (NaNs)

- Create test_meta variable with same rows and IDs as test dataset 
- Create Empty Columns: M1 and M2 (NaNs)

In [9]:
"""Create Train and Test Meta"""
train_meta = pd.DataFrame.copy(train)
train_meta["Model1_Predictions"] = np.nan
train_meta["Model2_Predictions"] = np.nan
print(train.shape)
train_meta.head()

(100, 5)


,Sepal Length,Sepal Width,Petal Length,Petal Width,Class,Model1_Predictions,Model2_Predictions
69,5.6,2.5,3.9,1.1,1,NaN,NaN
135,7.7,3.0,6.1,2.3,2,NaN,NaN
56,6.3,3.3,4.7,1.6,1,NaN,NaN
80,5.5,2.4,3.8,1.1,1,NaN,NaN
123,6.3,2.7,4.9,1.8,2,NaN,NaN


In [10]:
test_meta = pd.DataFrame.copy(test)
test_meta["Model1_Predictions"] = np.nan
test_meta["Model2_Predictions"] = np.nan
print(test.shape)
test_meta.head()

(50, 4)


,Sepal Length,Sepal Width,Petal Length,Petal Width,Model1_Predictions,Model2_Predictions
114,5.8,2.8,5.1,2.4,NaN,NaN
62,6.0,2.2,4.0,1.0,NaN,NaN
33,5.5,4.2,1.4,0.2,NaN,NaN
107,7.3,2.9,6.3,1.8,NaN,NaN
7,5.0,3.4,1.5,0.2,NaN,NaN


### Part 2

For each train fold: combine the other 4 into a training set and predict on each fold i 
- train fold 1 = 2,3,4,5 (Predict on fold 1)
- train fold 2 = 1,3,4,5 (Predict on Fold 2)
- train fold 3 = 1,2,4,5 (Predict on Fold 3)
- train fold 4 = 1,2,3,5 (Predict on Fold 4)
- train fold 5 = 1,2,3,4 (Predict on Fold 5)

In [12]:
from sklearn.model_selection import KFold
kf = KFold(n_splits = 5, shuffle=False)
for train_index, test_index in kf.split(train):
    print("TRAIN:", train_index, "TEST:", test_index, "\n")

TRAIN: [20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44
 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69
 70 71 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94
 95 96 97 98 99] TEST: [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19] 

TRAIN: [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 40 41 42 43 44
 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69
 70 71 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94
 95 96 97 98 99] TEST: [20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39] 

TRAIN: [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24
 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 60 61 62 63 64 65 66 67 68 69
 70 71 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94
 95 96 97 98 99] TEST: [40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59] 

TRAIN: [ 0  1  2  3  4  5  6  7  8

In [20]:
"""Self Created Cross Validation"""
"""Copy and Paste Output"""
j = 1
for train_index, test_index in kf.split(train):
    print ('train_fold{} = train[{}:{}]'.format(j, 0, max(train_index) + 1))
    print ('train_fold{} = train_fold{}.drop(train_fold{}.index[{}:{}])'.format(j, j, j, min(test_index), max(test_index) + 1))
    print ('test_fold{} = train[{}:{}]'.format(j, min(test_index), max(test_index) + 1), "\n")
    j += 1

train_fold1 = train[0:100]
train_fold1 = train_fold1.drop(train_fold1.index[0:20])
test_fold1 = train[0:20] 

train_fold2 = train[0:100]
train_fold2 = train_fold2.drop(train_fold2.index[20:40])
test_fold2 = train[20:40] 

train_fold3 = train[0:100]
train_fold3 = train_fold3.drop(train_fold3.index[40:60])
test_fold3 = train[40:60] 

train_fold4 = train[0:100]
train_fold4 = train_fold4.drop(train_fold4.index[60:80])
test_fold4 = train[60:80] 

train_fold5 = train[0:80]
train_fold5 = train_fold5.drop(train_fold5.index[80:100])
test_fold5 = train[80:100] 



In [21]:
train_fold1 = train[0:100]
train_fold1 = train_fold1.drop(train_fold1.index[0:20])
test_fold1 = train[0:20] 

train_fold2 = train[0:100]
train_fold2 = train_fold2.drop(train_fold2.index[20:40])
test_fold2 = train[20:40] 

train_fold3 = train[0:100]
train_fold3 = train_fold3.drop(train_fold3.index[40:60])
test_fold3 = train[40:60] 

train_fold4 = train[0:100]
train_fold4 = train_fold4.drop(train_fold4.index[60:80])
test_fold4 = train[60:80] 

train_fold5 = train[0:80]
train_fold5 = train_fold5.drop(train_fold5.index[80:100])
test_fold5 = train[80:100] 

In [41]:
for j in [1,2,3,4,5]:
    print("print(train_fold{}.shape)".format(j))
    print("print(test_fold{}.shape)".format(j), "\n")

print(train_fold1.shape)
print(test_fold1.shape) 

print(train_fold2.shape)
print(test_fold2.shape) 

print(train_fold3.shape)
print(test_fold3.shape) 

print(train_fold4.shape)
print(test_fold4.shape) 

print(train_fold5.shape)
print(test_fold5.shape) 



In [42]:
print(train_fold1.shape)
print(test_fold1.shape) 

print(train_fold2.shape)
print(test_fold2.shape) 

print(train_fold3.shape)
print(test_fold3.shape) 

print(train_fold4.shape)
print(test_fold4.shape) 

print(train_fold5.shape)
print(test_fold5.shape) 

(80, 5)
(20, 5)
(80, 5)
(20, 5)
(80, 5)
(20, 5)
(80, 5)
(20, 5)
(80, 5)
(20, 5)


### Part 3
- Use Model M1 to train on each train fold i and predict the excluded fold and store the values back into M1 for its fold ID
- Do the same with M2

In [ ]:
"""Instantiate Models"""
from 

### Part 4

Fit m1 Model to train_meta
- Put new predictions into train_meta m1 column
Fit m2 Model to train_meta
- Put new predictions into train_meta m2 column

### Part 5
- Predict m1 on test_meta
- Predict m2 on test_meta

Part 4
Use another model (S) and use train_meta to make final predictions on test_meta 

In [ ]:
# from sklearn.model_selection import KFold
# kf = KFold(n_splits=5, shuffle=False)
# for train, test in kf.split(data):
#     print("%s %s" % (train, test))

### KNN 

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier()
knn.fit(X_train, target_train)
target_pred = knn.predict(X_test)
print(round(accuracy(target_test, target_pred)*100, 2), "% Accuracy")

In [ ]:
knn = KNeighborsClassifier(n_neighbors = 10)
results = cross_val_score(knn, X, target, cv=5)
print(results.mean())

### Bagging

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import BaggingClassifier
est = KNeighborsClassifier(n_neighbors = 10)
bag = BaggingClassifier(base_estimator=est, 
                        n_estimators = 100, 
                        max_samples = 0.35,
                        max_features = 4,
                        random_state = 1, 
                        oob_score = True)
bag.fit(X_train, target_train)

print(bag.score(X_test, target_test))
print(bag.oob_score_)
print(bag.score(X_train, target_train))

In [ ]:
est = KNeighborsClassifier(n_neighbors = 10)
bag = BaggingClassifier(base_estimator=est, 
                        n_estimators = 100, 
                        max_samples = 0.35,
                        max_features = 4,
                        random_state = 1, 
                        oob_score = True)
results = cross_val_score(bag, X, target, cv=5)
print(results.mean())

### Random Forest

In [ ]:
from sklearn.ensemble import RandomForestClassifier
est = RandomForestClassifier()
rf = RandomForestClassifier(n_estimators = 100, oob_score = True)
rf.fit(X_train, target_train)

print(rf.score(X_test, target_test))
print(rf.oob_score_)
print(rf.score(X_train, target_train))

In [ ]:
est = RandomForestClassifier()
rf = RandomForestClassifier(n_estimators = 10)
results = cross_val_score(rf, X, target, cv=5)
print(results.mean())

### AdaBoost

In [ ]:
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier

est = AdaBoostClassifier()
model = RandomForestClassifier()
ada = AdaBoostClassifier(base_estimator = model, n_estimators = 100)
ada.fit(X_train, target_train)
print(ada.score(X_test, target_test))
print(ada.score(X_train, target_train))

In [ ]:
est = RandomForestClassifier()
ada = AdaBoostClassifier(base_estimator = est, n_estimators = 10)
results = cross_val_score(ada, X, target, cv=5)
print(results.mean())

### Gradient Tree Boosting

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier

est = GradientBoostingClassifier()
gbc = GradientBoostingClassifier(n_estimators = 100)
gbc.fit(X_train, target_train)
print(gbc.score(X_test, target_test))
print(gbc.score(X_train, target_train))

In [ ]:
gbc = GradientBoostingClassifier(n_estimators = 100)
results = cross_val_score(gbc, X, target, cv=5)
print(results.mean())

### Voting Classifier

In [ ]:
>>> from sklearn.model_selection import cross_val_score
>>> from sklearn.linear_model import LogisticRegression
>>> from sklearn.naive_bayes import GaussianNB
>>> from sklearn.ensemble import RandomForestClassifier
>>> from sklearn.ensemble import VotingClassifier

>>> clf1 = LogisticRegression(random_state = 1)
>>> clf2 = RandomForestClassifier(random_state = 1)
>>> clf3 = GaussianNB()
>>> clf4 = KNeighborsClassifier(n_neighbors = 8)
>>> eclf = VotingClassifier(estimators=[('lr', clf1), ('rf', clf2), ('gnb', clf3), ('knn', clf4)], voting='hard')


for clf, label in zip([clf1, clf2, clf3, clf4, eclf], ['Logistic Regression', 'Random Forest', 'Naive Bayes', 'KNN', 'Ensemble']):
    scores = cross_val_score(clf, X, target, cv=5, scoring='accuracy')
    print("Accuracy: %0.2f, (+/- %0.2f), [%s]" % (scores.mean(), scores.std(), label))

In [ ]:
estimator = bag
estimator.fit(X_train, target_train)
target_pred = estimator.predict(X_test)
print(round(accuracy(target_test, target_pred)*100, 2), "% Accuracy")

In [ ]:
actual = pd.DataFrame(target_test)
actual = actual.reset_index(drop=True)
actual.head()

In [ ]:
predictions = pd.DataFrame(target_pred)
predictions.columns = ["Predictions"]
predictions.head()

In [ ]:
pd.DataFrame.join(actual, predictions)